In [67]:
from csv import reader, DictReader, DictWriter
from langdetect import detect
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import re
import cld2

clean_up_re = re.compile('(\[[0-9]{4,4}-[0-9]{2,2}-[0-9]{2,2}\])? *[0-9]*(Food ordered:)?')

def clean_comment(comment):
    m = clean_up_re.match(comment)
    if m:
        comment = comment[m.span()[1]:]
    comment = comment.strip()
    return comment
    
def get_simple_label(label):
    return label.split(" ")[0]

def get_labels(el):
    labels = []
    for i in range(3):
        label = el[f'Topic {i+1}']
        #         print(label)
        if label:
            labels.append(get_simple_label(label))
    return labels
        
def mylang_detect(text):
    try:
        lang = detect(text)
    except:
        lang = "en"
    if lang not in ['en', 'de']:
        lang = cld2.detect(text)[2][0][1]
    if lang not in ['en', 'de']:
        lang = "en"
    return lang

In [48]:
csvdict = DictReader(open("/Users/aliosha/Downloads/Tagging.csv"), dialect='excel')

In [83]:
csvdict = DictReader(open("/Users/aliosha/Downloads/emails.csv"), dialect='excel')

In [85]:
email_list = []
for i, e in enumerate(csvdict):
    email_list.append(e)
    if i > 10000:
        break


In [87]:
email_list[1000:1010]

[OrderedDict([('file',
               '9KQIX5rRWW4NXJjIWU9+KoXt3NBtMaZBPOavx4Xg1FqBPlbVUHHNRKT3Q0lfUo3R3pHIfvHqBUE+'),
              ('message', None)]),
 OrderedDict([('file',
               '2KEtkZqxbyh1KyLgj2qjqkbPH8vbrWsZaaicNdCGGdw4lRsYI4rYu9QV7MMR86iseGFViGTz1xVm'),
              ('message', None)]),
 OrderedDict([('file',
               'GUbTuUHHFaKKM5asqwTG4kYlTWz4QVk8XaOqrx9ugz/38WscTKGfau09q1/Bkkh8WaPk8/boAfp5'),
              ('message', None)]),
 OrderedDict([('file',
               'i03sJbmh8W/+Shap/wBsf/RKVyRHeut+Ln/JQtU/7Y/+iUrlPSrh8KOobnI6gV6H8LbsS2N3YyfM'),
              ('message', None)]),
 OrderedDict([('file',
               'AwI+h4NefH2AroPh7eiw8QxByAsxKMKmotB3Eu9O/srxjHbuAE88EfjXr6Kfl9cCuS+IGiS3H2fU'),
              ('message', None)]),
 OrderedDict([('file',
               'bKMtPC65x3FdRaSO9jFJIpV/LBIPXOK5JO6QjyfWNM+0eNGtIBxJMOnp1NdN8R5EtNDtrJCBubp7'),
              ('message', None)]),
 OrderedDict([('file',
               'AVb8J6PJJqt3rF5GVZpCsW7sOlcr8RdQ+160Y

In [49]:
corpus = []
for el in csvdict:
    comment = clean_comment(el['comment'])
    labels = get_labels(el)
    score = el.get('score')
    if comment and labels:
        item = {"comment":comment,
               "labels":labels,
               "score":score}
        corpus.append(item)

In [50]:
len(corpus)

1083

In [51]:
corpus[:10]

[{'comment': "We ordered two curries and one soup. The curries arrived in bowls and were already mixed with the rice, which gave them the consistency of a watery soup. This wouldn't have been a big deal if they'd at least tasted good. But they didn't. The soup tasted so bad we simply threw it away.",
  'labels': ['PROD'],
  'score': 'Negative'},
 {'comment': 'Tuna Lemon Sandwich was a fairly small one – for the regular price way too small. Bread was already very dry on one side, felt like from the day before. Not inviting to order again.',
  'labels': ['PRICE', 'PROD', 'QA'],
  'score': 'Negative'},
 {'comment': 'Had a chicken burrito for lunch and it was the bomb!',
  'labels': ['PROD'],
  'score': 'Positive'},
 {'comment': 'one salad was missing', 'labels': ['QA'], 'score': 'Positive'},
 {'comment': 'falscher Burrito', 'labels': ['QA'], 'score': 'Positive'},
 {'comment': 'too late', 'labels': ['DEL'], 'score': 'Neutral / mixed'},
 {'comment': 'The wrap finally arrived, soggy and mush

In [52]:
import textacy
import spacy
import textblob
import textblob_de

In [53]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [54]:
analyzer = SentimentIntensityAnalyzer()

In [55]:
analyzer.polarity_scores("hey")

{'compound': 0.0, 'neg': 0.0, 'neu': 1.0, 'pos': 0.0}

In [56]:
analyzer = SentimentIntensityAnalyzer()

def get_sentiment(comment):
    sentiment_values = analyzer.polarity_scores(comment)

    if sentiment_values['compound'] > 0.5:
        return 'pos'
    elif sentiment_values['compound'] < -0.5:
        return 'neg'
    else:
        return 'neu'

    
# test_score = {'compound': 0.9551, 'neg': 0.0, 'neu': 0.645, 'pos': 0.355}
# get_sentiment(test_score)


In [62]:
textblob.TextBlob("not a kill review").sentiment

Sentiment(polarity=0.0, subjectivity=0.0)

In [68]:
def get_emotion_polarity(text, lang=None):
    if not lang:
        lang = mylang_detect(text)
    if lang == "en":
        tb = textblob.TextBlob(text)
    else:
        tb = textblob_de.TextBlobDE(text)
#     ts = doc_text[i]
#     em = textacy.lexicon_methods.emotional_valence(ts)
    return tb.sentiment.polarity


def get_emotion(text, lang=None):
    polarity = get_emotion_polarity(text, lang)
    if polarity < -0.05:
        emotion = "negative"
    elif polarity > 0.05:
        emotion = "positive"
    else:
        emotion = "neutral"
    return emotion


def create_corpus(csvdict):
    corpus = []
    for el in csvdict:
        comment = clean_comment(el['comment'])
        labels = get_labels(el)
        if comment:
            item = {"comment":comment,
                   "orig_labels":labels,
                   "orig_posit":el['+ / -']}
            corpus.append(item)
    return corpus


def predict_labels(text):
    return get_labels_from_dict(magpie.predict_from_text(text))

In [69]:
csvdict = DictReader(open("/Users/aliosha/Downloads/Tagging.csv"), dialect='excel')
my_dict = list(csvdict)
len(my_dict)

1276

In [70]:
element = my_dict[10]
element

OrderedDict([('id', 'int000011'),
             ('date', '2017-08-03'),
             ('brand', 'EarlsDeli'),
             ('score', 'Neutral / mixed'),
             ('source', 'internal'),
             ('channel', 'NA'),
             ('location', 'Xberg'),
             ('comment', 'wrong sandwich'),
             ('+ / -', 'negative'),
             ('Topic 1', 'QA –\xa0Missing or wrong products / ingredients'),
             ('Topic 2', ''),
             ('Topic 3', ''),
             ('Questions', '')])

In [71]:
corpus = create_corpus(my_dict)
corpus[:10]

[{'comment': "We ordered two curries and one soup. The curries arrived in bowls and were already mixed with the rice, which gave them the consistency of a watery soup. This wouldn't have been a big deal if they'd at least tasted good. But they didn't. The soup tasted so bad we simply threw it away.",
  'orig_labels': ['PROD'],
  'orig_posit': 'negative'},
 {'comment': 'Tuna Lemon Sandwich was a fairly small one – for the regular price way too small. Bread was already very dry on one side, felt like from the day before. Not inviting to order again.',
  'orig_labels': ['PRICE', 'PROD', 'QA'],
  'orig_posit': 'negative'},
 {'comment': 'Had a chicken burrito for lunch and it was the bomb!',
  'orig_labels': ['PROD'],
  'orig_posit': 'positive'},
 {'comment': 'one salad was missing',
  'orig_labels': ['QA'],
  'orig_posit': 'negative'},
 {'comment': 'falscher Burrito',
  'orig_labels': ['QA'],
  'orig_posit': 'negative'},
 {'comment': 'too late', 'orig_labels': ['DEL'], 'orig_posit': 'negat

In [72]:
for el in corpus:
    if el.get('orig_posit') not in ['positive', 'negative']:
        print(el.get('orig_posit'), get_sentiment(el['comment']), get_emotion(el['comment']), el['comment'])


n/a neu neutral Food Ordered:
n/a neu neutral (ops, we were hungry and are not foodies, we forgot to take pictures... next time, promise)
n/a neu neutral Orded #36787
n/a neu neutral * We got called letting us know that our payment option (cash, €50 bill) wouldn't work, also there was no change. If cash is not an option, maybe the checkbox for it shouldn't be visible?
n/a neu neutral Yellow Pizza with Huhnchen
n/a neu negative Hey green gurus,
n/a neu negative I just ordered the Veggie Thunder Quesadilla with:  Black Beans, extra cheddar, guacamole.
n/a neu positive Liebe GreenGurus,
n/a pos positive POSITIVE:
n/a pos positive Positive
n/a neu negative Hello Green Gurus Team,
n/a neu neutral Hallo Greengurus,
n/a neu neutral https://gurucollective.zendesk.com/agent/tickets/8574
n/a neu positive **There is a glitch in your ordering system!!--->> I was able to order a soda as an add-on and was not charged for it.  Also I could add bread and croutons for no extra charge (not sure if there

n/a neg negative Ihr liefert leider nicht mehr zu meiner Wohnung in 13355. Die neuen Preise sind deutlich über einem normalen Lunch Budget in Berlin. Das ist sehr schade.
n/a pos positive alles super
n/a neu neutral Wir haben schon einmal bei euch bestellt und waren zufrieden.
n/a neu neutral Nicht jede Woche gibt es Bowls, die eine 10 verdienen, aber eine 8 doch allemal :)
n/a neu negative Liefergebiet wurde zu stark eingeschränkt
n/a neu neutral !
n/a neu neutral Übersichtlich, leichter Bestellvorgang mit dem Handy
n/a neu neutral Hey, ich habe mit meinem Mann zusammen schon desöftern bestellt bei euch.
n/a neg negative Ich habe erst versucht im Safari Browser 10.1.2 auf Mac OSX zu bestellen, konnte aber im Regstrierungsformular meine Daten nicht richtig eingeben weil der Inhalt der Eingabefelder unlesbar war. Dann habe ich es mit Chrome versucht und da hat es problemlos geklappt. War halt ziemlich umständlich.
n/a neu neutral Hups wollte 10 klicken
n/a neu neutral Weil es sehr wahrs

In [76]:
analyzer.polarity_scores("wrong sandwich")

{'compound': -0.4767, 'neg': 0.756, 'neu': 0.244, 'pos': 0.0}

In [79]:
def translate(rev):
    if rev == "pos":
        return "positive"
    if rev == "neg":
        return "negative"
    return "neutral"

def are_equal(rev1, rev2):
    if translate(rev1) == rev2:
        return True
    return False
        

for el in corpus:
    if not are_equal(get_sentiment(el['comment']), get_emotion(el['comment'])):
        print(el.get('orig_posit'), get_sentiment(el['comment']), get_emotion(el['comment']), el['comment'])


negative pos negative Tuna Lemon Sandwich was a fairly small one – for the regular price way too small. Bread was already very dry on one side, felt like from the day before. Not inviting to order again.
positive pos negative Had a chicken burrito for lunch and it was the bomb!
negative neu negative one salad was missing
negative neu negative falscher Burrito
negative neu negative too late
negative neu negative wrong sandwich
negative neu negative Mild Massaman was cold, had no sweet potato, hardly any rice. Seemed to be peanut butter sauce with green beans only.
negative neu negative The customer ordered a vegan burrito bowl and some extras. The order got delivered after more than one hour. The bowl was half empty, beans were missing as well as at least three extras.
negative neu negative ein Salat wurde vergessen
negative neg positive Only thing: the customer complaints about the packaging of the burritos, which is not environmental friendly / sustainable.
negative neu negative The c

negative neu positive Amigo Bowl: Dressing isn't that great, very sour and watery. The bowl would have been nicer without it.
negative neg positive Guacemole wurde leider vergessen. Als regulärer Kunde wäre ich insgesamt unzufrieden gewesen. Portion etwas klein. Die Aufteilung der Take-Away Schale ist etwas unvorteilhaft, da in der zweiten Kammer nur etwas Eisbergsalat und ein paar Tacos sind, diese aber trotzdem 40% der Schale ausmachen.
positive neu positive Leckere Chipotle Majo! Essen kam sehr schnell von Refael. Hab 5 Sterne bei Lieferando gegeben.
negative neu positive I ordered the Antipasti Wrap: It mostly consisted of spinach. There was no mozzarella in it instead I found some gouda kind of cheese. Plus there was only like half of a little tomato and artichoke in it. Wouldn't order it again.
n/a neu negative Hello Green Gurus Team,
negative neu negative Cold, slow and not enough, bu packaging was better than before at least
negative neu negative Took 1 hour to get delivery and

negative neu negative Got the yellow beets and avocado sandwich and it was dry, hastily put together and small. Did it even have beets? When I looked at it, it didn't seem like something that invited me to eat it; and upon eating it, the taste didn't move me to keep going either.
negative neu negative Guys, when I had the Tikka Masala Curry on the day we launched IndiaLab, I thought it would become my new favourite dish in all of GuruCollective. I had it again today and I'm really disappointed. It's a bit tangy / sour, the sauce is not as flavourful anymore and the tofu tasted like nothing so I didn't even finish it. I wouldn't order from this restaurant again. (Rike)
n/a neu negative I ordered twice: Pizza White, Pizza Green & Pizza Funghi + Tonno Salad
negative neu negative Unfortunately, all pizzas were cold / seemed to be a bit older. , but: Pizza White - Bechamel, where are you? & Pizza Green, very dry and needed another portion of salt & pepper
positive neu positive Habe einen Sa

negative neu negative Pizza a but dry
negative pos negative eine falsche bowl wurde geliefert. in einem Salat fehlte Avocado. zudem gab es eine allergische Reaktion auf den sweet potato wrap (gegen angegebene Allergene: Weizen, Mandeln, besteht aber keine Allergie!)
negative neu positive [FOOD ISSUE] There was a piece of plastic on the pizza.. the pizza wasn't as warm compared to other deliveries, the Margarita was very basic the cheese melted as if it were in a microwave... The chicken salad with avocado was fresh and worth it's price.
negative neu negative I ordered the Tom Kha and the Mean Green Curry - unfortunately the soup was cold when it arrived, I ordered it with cashews, but only found three of them. Some more would have been nice.
negative neg neutral [FOOD ISSUE] Kunde war bei vorheriger Bestellung zunächst verwundert über das neue Brot – Weißbrot anstelle von "richtigem" Brot. Jetzt beschwert er sich über Qualität des Weißbrots (loch in Brot): "Ich kann verstehen das man e

negative neu negative [FOOD ISSUE] -The figs are too big and also still had part of the stem attached to it, making them difficult to eat
negative neu positive [FOOD ISSUE] [PACKAGING ISSUE] Wrapping warm sandwiches in tin foil is really bad: ciabatta bread was incredibly soggy
negative neu negative [FOOD ISSUE] wakame salad: portion too small, customer upset, kitchen used big boxes instead of small ones for the usual amount of food so it looked less. had to refund the order.
negative neu negative [TIME ISSUE] [2017-11-12] 8 mins too late...
positive neu positive Eureka bowl: very good
negative neu negative [FOOD ISSUE] Vegan Waldorf Salad had a few bad grapes (brown squishy). Please check them before putting them in.
n/a neu positive Every thing went smooth today. Thanks a lot!
negative pos negative [FOOD ISSUE] [2017-11-05] Customer: "Edible but otherwise bad. The burritos taste anything but Mexican (or even Tex Mex), the salsa verde honestly tasted like it had tons of cumin (?!) ins

...

positive neu positive Best sandwiches in Kreuzberg for sure. Too bad there's no Pommes!
negative neg positive Weil die Bestellung heute 40 Minuten zu spät kam und ich einen Anruf erhielt, dass die Bestellung nur 15 Min zu spät kommt. Dann habe ich angerufen und mir wurde gesagt, dass ein neuer Lieferant unterwegs ist und im Büro angerufen hat, weil er uns nicht gefunden hat. Da finde ich es wichtig, dass die Mitarbeiter so geschult sind, dass sie bei Unklarheit wissen, was zutun ist. Sonst ist alles okay. Schönes Wochenende
negative neu positive The food just wasn't really good
positive neu positive lecker gesund und schnell. sehr freundliche lieferanten
positive neu positive Einfach lecker und gut.
n/a neu positive Bin mir unsicher, ob das Essen pünktlich ankommt, wenn gleich noch ein Tracking angeboten wird. Oder gibt es ein Zeitfenster? Das wäre nachvollziehbarer. Sonst allet schick!!!
positive neu positive Tolles gesundes Essen, dazu sehr lecker und schnell geliefert. Einziger Krit

positive neg positive Geschmacklich sehr gut und auch die umweltfreundliche Verpackung ist toll
positive neg positive Die burritos waren sehr lecker und reichhaltig -
negative neg neutral die nachos fand ich mit 3€ etwas teuer - und die dips dazu waren sehr spärlich..
negative neg positive Ich finde den Laden und das Essen cool. Leider war die Lieferung signifikant zu spät, das Essen nicht mehr Frisch und das Getränk hatte Zimmertemperatur. Kann aber jedem mal passieren und ist für uns kein Grund zum Absprung. Weiterhin viel Erfolg und viele Grüße!
positive neu positive Ich finde das Konzept der Office-Deals sehr innovativ - bequem einzeln vorbestellen (sodass nicht einer alles auslegen muss) und das Essen ist in jedem Fall pünktlich zur Mittagszeit da. :) Die Karte könnte etwas häufiger wechseln, sonst alles super.
positive neu positive Übersichtlich, schnell und lecker
negative neg neutral Es fehlte ein Gericht, die Fahrerin sprach nur englisch und wusste von nichts, das Restaurant w

negative neu negative Miesestes Preis-Leistungs-Verhältnis. Beilagen waren ohne jegliche Mühe zubereitet. , aber halsabschneiderisch. 7€ für ein Bun mit etwas Belag. Scheußlich.
positive neg neutral Die Burger dagegen waren lecker
negative neg positive Die Qualität rechtfertigt den Preis in keinster Weise. Sehr kleine Bürger mit sehr wenig drauf
negative neg positive Das Sandwich war okay, vom Preis Leistung Verhältnis aber total überteuert. Die Suppe war gut
negative neg neutral Kleine portion..vieeel zu teuer dafür was es war
n/a neu positive Toll, dass Ihr wieder liefert!
n/a neg neutral Gestrigen Order war eine Katastrophe und wurde reklamiert.
negative neu negative Essen kalt, Portion winzig und das Chili hat nach nichts geschmeckt. Alles im allen nie so enttäuscht worden im Preis Leistung Verhältnis
n/a pos neutral Service und Support war super. Nur beim Telefon geht nie jemand ran ;-)
n/a neu positive Top
positive neg neutral Essen war frisch.
n/a neu positive Übersichtlicher Sh

positive neg neutral Der Burrito war lecker,
negative neu negative allerdings nicht pikant wie eigentlich beschrieben und dazu fast kalt.
negative neg neutral Die Größe der Portion ist für den Preis ein Witz !!!
n/a neu negative Sorry guys, but this was nothing
n/a neu positive Site looks good so far
negative pos negative super schlechte pizza. dazu noch kalt
negative neg positive Wie bei bisher jeder Bestellung hat mal wieder was nicht gestimmt: Dieses Mal war kein Brot/Brötchen dabei, obwohl auf dem Bestellzettel auch vermerkt. (Es geht nicht um das Brot, sondern dass einfach jedes Mal was nicht passt.)
positive neg neutral Das Essen war lecker,
negative neu negative Kein Saltz im Essen und es schmeckt nicht gut
n/a neu positive Gut
positive neg neutral Er war lecker!
positive neg neutral war alles super-lecker
negative neg neutral der rest war leider unterirdisch
negative neg neutral - Das Essen kam nach 90 Minuten an (bisher die längste Lieferzeit unter der Woche bei einem Lieferdi

negative neg positive Aber man muss sie erst stehen lassen, weil sie zu kalt sind. Und die Dressings finde ich nicht so überzeugend.
positive neu positive netter Service, immer frisch und lecker sowie so.
positive neu positive Top Essen und Service
positive neg neutral Die besten Salate der Stadt! Danke Green-Gurus!
n/a neu positive Weil ich euch schon mehrfach weiter empfohlen habe ;)


In [81]:
t_doc = textacy.doc.Doc("Got the chicken burritos and quesadillas.")

In [82]:
textacy.preprocess_text("Got the chicken burritos and quesadillas.")

'Got the chicken burritos and quesadillas.'

In [ ]:
textacy

In [37]:
def get_labels_from_dict(labels, min_val=0.1):
    return_labels = []
    for label, val in labels:
#         print(label, val)
        if val > min_val:
            return_labels.append(label)
    return return_labels

In [42]:
# for el in corpus:
#     comment = el["comment"]
#     el["predict_labels"] = predict_labels(comment)
#     el["predict_posit"] = get_emotion(comment)
# corpus[:10]

In [45]:
with open('results.csv', 'w', newline='') as csvresults:
    csv_labels = ['comment', 'orig_labels', 'predict_labels', 'orig_posit', 'predict_posit']
    writer = DictWriter(csvresults, fieldnames=csv_labels)
    writer.writeheader()
    for el in corpus:
        writer.writerow(el)
